In [1]:
from collections import defaultdict

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from data import classification_dataset, train_test_split, kfolds
from layers import SeluConv3D, SeluDense
from plot import plot_slice, plot_volume_animation
from config import (
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_BIG_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_BIG_POS_TFRECORD,
    SPIE_SMALL_NEG_TFRECORD,
    SPIE_BIG_NEG_TFRECORD,
    SPIE_SMALL_POS_TFRECORD,
    SPIE_BIG_POS_TFRECORD,
    SMALL_PATCH_SHAPE,
    BIG_PATCH_SHAPE,
)

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [2]:
SEED = 5

In [3]:
lidc_dataset, lidc_samples = classification_dataset(
    LIDC_SMALL_NEG_TFRECORD,
    LIDC_BIG_NEG_TFRECORD,
    LIDC_SMALL_POS_TFRECORD,
    LIDC_BIG_POS_TFRECORD,
    return_size=True,
)
print(f"{lidc_samples = }")
lidc_dataset

lidc_samples = 754


<ShuffleDataset shapes: (((None, None, None, None), (None, None, None, None)), (1,)), types: ((tf.float32, tf.float32), tf.int8)>

In [4]:
def build_3d_cnn(dropout_rate=0.0):
    input_small = keras.Input(SMALL_PATCH_SHAPE, name="input_small")
    x_small = SeluConv3D(
        filters=32,
        kernel_size=3,
        name="small_selu_conv3d_1",
    )(input_small)
    x_small = keras.layers.MaxPool3D((1, 2, 2), name="small_maxpool_1")(x_small)
    x_small = SeluConv3D(
        filters=64,
        kernel_size=3,
        name="small_selu_conv3d_2",
    )(x_small)
    x_small = keras.layers.MaxPool3D((1, 2, 2), name="small_maxpool_2")(x_small)
    x_small = SeluConv3D(
        filters=128,
        kernel_size=3,
        name="small_selu_conv3d_3",
    )(x_small)
    x_small = keras.layers.MaxPool3D((1, 2, 2), name="small_maxpool_3")(x_small)
    x_small = SeluConv3D(
        filters=256,
        kernel_size=3,
        name="small_selu_conv3d_4",
    )(x_small)
    x_small = keras.layers.MaxPool3D((1, 1, 2), name="small_maxpool_4")(x_small)
    x_small = keras.layers.Flatten(name="flatten_small")(x_small)

    input_big = keras.Input(BIG_PATCH_SHAPE, name="input_big")
    x_big = keras.layers.MaxPool3D((2, 2, 2), name="big_maxpool_0")(input_big)
    x_big = SeluConv3D(
        filters=32,
        kernel_size=3,
        name="big_selu_conv3d_1",
    )(x_big)
    x_big = keras.layers.MaxPool3D((1, 2, 2), name="big_maxpool_1")(x_big)
    x_big = SeluConv3D(
        filters=64,
        kernel_size=3,
        name="big_selu_conv3d_2",
    )(x_big)
    x_big = keras.layers.MaxPool3D((1, 2, 2), name="big_maxpool_2")(x_big)
    x_big = SeluConv3D(
        filters=128,
        kernel_size=3,
        name="big_selu_conv3d_3",
    )(x_big)
    x_big = keras.layers.MaxPool3D((1, 2, 2), name="big_maxpool_3")(x_big)
    x_big = SeluConv3D(
        filters=256,
        kernel_size=3,
        name="big_selu_conv3d_4",
    )(x_big)
    x_big = keras.layers.MaxPool3D((1, 1, 2), name="big_maxpool_4")(x_big)
    x_big = keras.layers.Flatten(name="flatten_big")(x_big)

    x = keras.layers.concatenate([x_small, x_big], name="concatenate")
    #x = SeluDense(128, name="selu_dense")(x)
    #x = keras.layers.AlphaDropout(dropout_rate)(x)
    x = keras.layers.Dense(1, activation="sigmoid", name="final_dense")(x)

    cnn_3d = keras.Model(inputs=[input_small, input_big], outputs=x, name="3dcnn")

    return cnn_3d

In [5]:
learning_rate = 1e-5
val_perc = 0.1
patience = 30
batch_size = 16
dropout_rate = 0.5
metrics = [
    keras.metrics.AUC(name="auc"),
    keras.metrics.BinaryAccuracy(name="accuracy"),
]

In [ ]:
train_dataset, val_dataset = train_test_split(
    lidc_dataset, test_perc=val_perc, seed=SEED
)
val_dataset = val_dataset.batch(batch_size)
train_dataset = (
    train_dataset.cache()  # must be called before shuffle
    .shuffle(buffer_size=1024, reshuffle_each_iteration=True)
    .batch(batch_size)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

cnn = build_3d_cnn(dropout_rate)
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=keras.losses.MeanSquaredError(),
    metrics=metrics,
)
cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1000,
    verbose=1,
    callbacks=[
        keras.callbacks.ModelCheckpoint(
            "models/lidc-3d-cnn.h5",
            monitor="val_loss",
            save_best_only=True,
            verbose=1,
        ),
        keras.callbacks.EarlyStopping(
            monitor="val_loss",
            patience=patience,
            restore_best_weights=True,
        ),
    ],
)

In [ ]:
cnn.evaluate(val_dataset, return_dict=True)

In [6]:
spie_dataset, spie_samples = classification_dataset(
    SPIE_SMALL_NEG_TFRECORD,
    SPIE_BIG_NEG_TFRECORD,
    SPIE_SMALL_POS_TFRECORD,
    SPIE_BIG_POS_TFRECORD,
    return_size=True,
)
print(f"{spie_samples = }")
spie_dataset

spie_samples = 73


<ShuffleDataset shapes: (((None, None, None, None), (None, None, None, None)), (1,)), types: ((tf.float32, tf.float32), tf.int8)>

In [7]:
cnn = keras.models.load_model("models/lidc-3d-cnn.h5")
cnn.evaluate(spie_dataset.batch(1), return_dict=True)

73/73 [==============================] - 0s 4ms/step - loss: 0.3196 - auc: 0.6764 - accuracy: 0.5890


{'loss': 0.31961071491241455,
 'auc': 0.6764264702796936,
 'accuracy': 0.5890411138534546}

In [18]:
def build_pretrained_3d_cnn():
    pretrained_3d_cnn = keras.models.load_model("models/lidc-3d-cnn.h5")
    #for layer in pretrained_3d_cnn.layers:
    #    if "conv" in layer.name:
    #        layer.trainable = False
    return pretrained_3d_cnn

In [19]:
k = 3
learning_rate = 1e-5
val_perc = 0.1
patience = 30
num_epochs = 1000
batch_size = 8
dropout_rate = 0.5
metrics = [
    keras.metrics.AUC(name="auc"),
    keras.metrics.BinaryAccuracy(name="accuracy"),
]

In [20]:
lidc_mean_metrics = {
    f"{metric.name}": keras.metrics.Mean(name=f"mean_{metric.name}")
    for metric in metrics
}
wo_pt_mean_metrics = {
    f"{metric.name}": keras.metrics.Mean(name=f"mean_{metric.name}")
    for metric in metrics
}
w_pt_mean_metrics = {
    f"{metric.name}": keras.metrics.Mean(name=f"mean_{metric.name}")
    for metric in metrics
}
fold_id = 0
for trainval_dataset, test_dataset in tqdm(
    kfolds(k, spie_dataset, cardinality=spie_samples, seed=6), total=k
):
    print(f" {fold_id = } ".center(50, "="))

    test_dataset = test_dataset.batch(batch_size)
    train_dataset, val_dataset = train_test_split(trainval_dataset, test_perc=val_perc)
    val_dataset = val_dataset.batch(batch_size)
    train_dataset = (
        train_dataset.cache()  # must be called before shuffle
        .shuffle(buffer_size=1024, reshuffle_each_iteration=True)
        .batch(batch_size)
        .prefetch(tf.data.experimental.AUTOTUNE)
    )
    print(f"Train size: {sum(1 for _ in train_dataset.unbatch())}")
    print(f"Validation size: {sum(1 for _ in val_dataset.unbatch())}")
    print(f"Test size: {sum(1 for _ in test_dataset.unbatch())}")
    print()

    cnn = keras.models.load_model("models/lidc-3d-cnn.h5")
    test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=0)
    print("LIDC training only: ")
    for metric_name, metric_value in test_metrics.items():
        print(f"{metric_name}: {metric_value}")
        if metric_name in lidc_mean_metrics:
            lidc_mean_metrics[metric_name].update_state(metric_value)
    print("")

    cnn = build_3d_cnn(dropout_rate)
    cnn.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.MeanSquaredError(),
        metrics=metrics,
    )
    cnn.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=num_epochs,
        verbose=0,
        callbacks=[
            keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=patience,
                restore_best_weights=True,
            ),
        ],
    )
    test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=0)
    print("Without pretraining: ")
    for metric_name, metric_value in test_metrics.items():
        print(f"{metric_name}: {metric_value}")
        if metric_name in wo_pt_mean_metrics:
            wo_pt_mean_metrics[metric_name].update_state(metric_value)
    print("")

    cnn = build_pretrained_3d_cnn()
    cnn.compile(
        optimizer=keras.optimizers.Adam(learning_rate),
        loss=keras.losses.MeanSquaredError(),
        metrics=metrics,
    )
    cnn.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=num_epochs,
        verbose=0,
        callbacks=[
            keras.callbacks.EarlyStopping(
                monitor="val_loss",
                patience=patience,
                restore_best_weights=True,
            ),
        ],
    )
    test_metrics = cnn.evaluate(test_dataset, return_dict=True, verbose=0)
    print("With pretraining: ")
    for metric_name, metric_value in test_metrics.items():
        print(f"{metric_name}: {metric_value}")
        if metric_name in w_pt_mean_metrics:
            w_pt_mean_metrics[metric_name].update_state(metric_value)
    fold_id += 1

print(" average ".center(50, "="))
print("LIDC training only: ")
for metric_name, metric_value in lidc_mean_metrics.items():
    print(f"{metric_name}: {metric_value.result()}")
print("")
print("Without pretraining: ")
for metric_name, metric_value in wo_pt_mean_metrics.items():
    print(f"{metric_name}: {metric_value.result()}")
print("")

print("With pretraining: ")
for metric_name, metric_value in w_pt_mean_metrics.items():
    print(f"{metric_name}: {metric_value.result()}")

================== fold_id = 0 ===================
Train size: 45
Validation size: 4
Test size: 24

LIDC training only: 
loss: 0.274137407541275
auc: 0.7777777910232544
accuracy: 0.625

Without pretraining: 
loss: 0.19191725552082062
auc: 0.788194477558136
accuracy: 0.7916666865348816

With pretraining: 
loss: 0.20678023993968964
auc: 0.7777778506278992
accuracy: 0.75
================== fold_id = 1 ===================
Train size: 45
Validation size: 4
Test size: 24

LIDC training only: 
loss: 0.28158077597618103
auc: 0.7027972340583801
accuracy: 0.6666666865348816

Without pretraining: 
loss: 0.2362479418516159
auc: 0.6608391404151917
accuracy: 0.5833333134651184

With pretraining: 
loss: 0.1781003326177597
auc: 0.8391607999801636
accuracy: 0.7083333134651184
================== fold_id = 2 ===================
Train size: 44
Validation size: 4
Test size: 25

LIDC training only: 
loss: 0.39977383613586426
auc: 0.58441561460495
accuracy: 0.47999998927116394

Without pretraining: 
loss: 0.